In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## 数据预处理

In [20]:
# 加载数据
file_path = r"F:\cache_data\frequency_filter\dy\scaler_csv\dy_stander_filter_all_type.csv"
data = pd.read_csv(file_path)

# 分离特征和标签
X = data.drop(['TL', 'YL', 'TS', 'TZ'], axis=1)
y_tl = LabelEncoder().fit_transform(data['TL'])
y_yl = LabelEncoder().fit_transform(data['YL'])
y_ts = LabelEncoder().fit_transform(data['TS'])
y_tz = LabelEncoder().fit_transform(data['TZ'])
y = np.vstack((y_tl, y_yl, y_ts, y_tz)).T

In [22]:
# 标准化特征
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.2, random_state=42)

# 转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 创建DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## 模型构建

In [43]:
class SoilClassifier(nn.Module):
    def __init__(self):
        super(SoilClassifier, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc_tl = nn.Linear(64, len(np.unique(y_tl)))
        self.fc_yl = nn.Linear(64, len(np.unique(y_yl)))
        self.fc_ts = nn.Linear(64, len(np.unique(y_ts)))
        self.fc_tz = nn.Linear(64, len(np.unique(y_tz)))
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        tl_pred = self.fc_tl(x)
        yl_pred = self.fc_yl(x)
        ts_pred = self.fc_ts(x)
        tz_pred = self.fc_tz(x)
        return tl_pred, yl_pred, ts_pred, tz_pred

model = SoilClassifier()


## 训练模型

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

def train_model(model, train_loader, criterion, optimizer, epochs=30000):
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = sum(criterion(output, label) for output, label in zip(outputs, labels.T))
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train_model(model, train_loader, criterion, optimizer)


## 评估模型

In [25]:
def evaluate_model(model, test_loader):
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            for output, label in zip(outputs, labels.T):
                _, predicted = torch.max(output, 1)
                total_correct += (predicted == label).sum().item()
                total_samples += label.size(0)
    accuracy = total_correct / total_samples
    print(f'Total Accuracy: {accuracy}')

evaluate_model(model, test_loader)


Total Accuracy: 0.4559155990957046
